In [5]:
from binance.client import Client
import os
import json
from binance import ThreadedWebsocketManager
import numpy as np

In [6]:
with open('../config/key.json') as j:
    keys = json.load(j)

In [7]:
data = {
    "price_buy":[],
    "price_sell":[],
    "price":0,
    "status":True,
    "sell":0,
    "sw":True,
    "first":True,
    "symbol":"ETHBUSD",
    "symbol_play": "ETH",
    "back_price": np.array([]),
    "price_play":15
}

In [8]:
client = Client(keys['api_key'], keys['api_secret'])
dict_data = {'error':False}
bsm = ThreadedWebsocketManager()

In [11]:
fees = client.get_trade_fee(symbol='ETHBUSD')
fees

[{'symbol': 'ETHBUSD', 'makerCommission': '0', 'takerCommission': '0.001'}]

In [ ]:
sell_price = client.get_asset_balance(asset=data['symbol_play']).get('free')
order_sell = client.order_market_sell(symbol=data['symbol'],quantity=round(float(sell_price),4))

In [5]:

def btc_trade_history(msg):

    if data['status'] != True:
        print('save data')
        with open("data.json", "w+") as outfile:
            data['back_price'] = data['back_price'].tolist()
            json.dump(data, outfile)
        bsm.stop()
        
    if msg['e'] != 'error':
        if data['first']:
            data['back_price'] = np.append(data['back_price'],float(msg['c']))
            if len(data['back_price']) == 10:
                print('initial app and prepare data')
                print("msg:",msg)
                data["price"] = msg['c']
                data['first'] = False
                data['sw'] = False
        else:
            
            # delete first index in array
            data['back_price'] = np.delete(data['back_price'], 0)
#           add new data to array
            data['back_price'] = np.append(data['back_price'],float(msg['c']))

            # print("std:",np.std(data['back_price']))
            std = np.std(data['back_price'])
            
#             len_bprice = len(data['back_price'])
#             positive_prob = 0
#             for i in range(len_bprice):
#                 cur_var = data['back_price'][i]
#                 try:
#                     curp1_var = data['back_price'][i+1]
#                     if cur_var < curp1_var:
#                         positive_prob+=1 
#                 except IndexError:
#                     break
                
#             # positive_prob = np.sum(prob_back_number)
#             negative_prob = len_bprice - positive_prob
#             check_prob = True # true:positive, false:nagative
#             if positive_prob < negative_prob:
#                 check_prob = False
            
#             print('check_prob:',check_prob)
            if data['sw']:
                data['price'] = float(data['price'])
                msg['c'] = float(msg['c'])
                if float(msg['c']) > float(data['price']):
                    data['price'] = max(data['price'],float(msg['c']))
                    

                if float(msg['c']) < data['price']:
                    max_var = max(data['price'],msg['c'])
                    min_var = min(data['price'],msg['c'])
                    percent_sell = ((max_var-min_var)/min_var)*100
                    # print("percent sell:",percent_sell)
                    if percent_sell > 0.12 and msg['c'] > data['sell']:
                        # sell
                        sell_price = client.get_asset_balance(asset=data['symbol_play']).get('free')
                        print('sell price',round(float(sell_price),3))
                        print('type',type(round(float(sell_price),3)))
                        # order_sell = client.order_market_sell(symbol=data['symbol'],quantity=round(float(sell_price),4))
                        # print('order_sell:',order_sell)
                        data["price_sell"].append(msg['c'])
                        # save price sell
                        data['price'] = msg['c']
                        print('sell:',data['price'])
                        # change status sw to False
                        data['sw'] = False

                    # if data['price'] >= data['price_buy'][-1]: 
                    #     max_sell_acc = max(data["price"],msg['c'])
                    #     min_sell_acc = min(data["price"],msg['c'])
                    #     percent_sell_acc = ((max_sell_acc-min_sell_acc)/min_sell_acc)*100
                    #     if percent_sell_acc >=0.001 and msg['c'] > data['sell'] and std < 0.51:
                    #         # sell_price = data["earn_coin"]+round((msg['c'] -  data['price_buy'][-1]),2)
                    #         sell_price = client.get_asset_balance(asset=data['symbol_play']).get('free')
                    #         print('sell price',round(float(sell_price),3))
                    #         print('type',type(round(float(sell_price),3)))
                    #         # order_sell = client.order_market_sell(symbol=data['symbol'],quantity=round(float(sell_price),4))
                    #         # print('order_sell:',order_sell)
                    #         data["price_sell"].append(msg['c'])
                    #         data['price'] = msg['c']
                    #         print('sell:',data['price'])
                    #         data['sw'] = False

            else:
                msg['c'] = float(msg['c'])
                data['price'] = float(data['price'])
                if data['price'] > msg['c']:
                    data['price'] = min(data['price'],msg['c'])

                if msg['c'] > data['price']:
                    max_var = max(data['price'],msg['c'])
                    min_var = min(data['price'],msg['c'])
                    percent_buy = ((max_var-min_var)/min_var)*100
                    # print("percent buy:",percent_buy)
                    if percent_buy > 0.12:
                        buy_coin = round(data["price_play"]/msg['c'],4)
                        print("buy_coin:",buy_coin)
                        print('type buy_coin',type(buy_coin))
                        # order_buy = client.order_market_buy(symbol=data['symbol'],quantity=buy_coin)
                        # print('order_buy:',order_buy)
                        data["price_buy"].append(msg['c'])
                        print('buy:',float(msg['c']))
                        data["price"] = float(msg['c'])
                        data['price'] = msg['c']
                        data['sell'] = msg['c']
                        data['sw'] = True
                    
        dict_data['last'] = msg['c']
        dict_data['bid'] = msg['b']
        dict_data['last'] = msg['a']
        dict_data['error'] = False
    else:
        dict_data['error'] = True


In [7]:
bsm.stop()

In [6]:
bsm.start()
print('starting ...... ')
bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol=data["symbol"])

starting ...... 


'ethbusd@ticker'

initial app and prepare data
msg: {'e': '24hrTicker', 'E': 1656084235765, 's': 'ETHBUSD', 'p': '93.79000000', 'P': '8.538', 'w': '1156.17994354', 'x': '1098.26000000', 'c': '1192.26000000', 'Q': '0.45910000', 'b': '1192.26000000', 'B': '1.57600000', 'a': '1192.27000000', 'A': '9.54940000', 'o': '1098.47000000', 'h': '1225.30000000', 'l': '1081.77000000', 'v': '425854.69470000', 'q': '492364656.87654400', 'O': 1655997835593, 'C': 1656084235593, 'F': 317523265, 'L': 318120966, 'n': 597702}


In [9]:
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
timestamp

1502928000000

In [24]:
bars = client.get_historical_klines('ETHBUSD', client.KLINE_INTERVAL_1DAY,"24 May, 2022")

In [26]:
len(bars)

32